<a href="https://colab.research.google.com/github/Madushani-Weerasekara/D10_Fake-News-Classifier-Using-LSTM/blob/main/D10_Fake_News_Classifier_Using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Fake News Classifier Using LSTM

In [98]:
import pandas as pd


In [99]:
df = pd.read_csv('/content/FakeNewsNet.csv')

In [100]:
df.head()

,title,news_url,source_domain,tweet_num,real
0,Kandi Burruss Explodes Over Rape Accusation on...,http://toofab.com/2017/05/08/real-housewives-a...,toofab.com,42,1
1,People's Choice Awards 2018: The best red carp...,https://www.today.com/style/see-people-s-choic...,www.today.com,0,1
2,Sophia Bush Sends Sweet Birthday Message to 'O...,https://www.etonline.com/news/220806_sophia_bu...,www.etonline.com,63,1
3,Colombian singer Maluma sparks rumours of inap...,https://www.dailymail.co.uk/news/article-33655...,www.dailymail.co.uk,20,1
4,Gossip Girl 10 Years Later: How Upper East Sid...,https://www.zerchoo.com/entertainment/gossip-g...,www.zerchoo.com,38,1


In [101]:
df.shape

(23196, 5)

In [102]:
df.isnull().sum()

title              0
news_url         330
source_domain    330
tweet_num          0
real               0
dtype: int64

In [103]:
##Drop Null values
df.dropna()

,title,news_url,source_domain,tweet_num,real
0,Kandi Burruss Explodes Over Rape Accusation on...,http://toofab.com/2017/05/08/real-housewives-a...,toofab.com,42,1
1,People's Choice Awards 2018: The best red carp...,https://www.today.com/style/see-people-s-choic...,www.today.com,0,1
2,Sophia Bush Sends Sweet Birthday Message to 'O...,https://www.etonline.com/news/220806_sophia_bu...,www.etonline.com,63,1
3,Colombian singer Maluma sparks rumours of inap...,https://www.dailymail.co.uk/news/article-33655...,www.dailymail.co.uk,20,1
4,Gossip Girl 10 Years Later: How Upper East Sid...,https://www.zerchoo.com/entertainment/gossip-g...,www.zerchoo.com,38,1
...,...,...,...,...,...
23191,Pippa Middleton wedding: In case you missed it...,https://www.express.co.uk/news/royal/807049/pi...,www.express.co.uk,52,1
23192,Zayn Malik & Gigi Hadid’s Shocking Split: Why ...,hollywoodlife.com/2018/03/13/zayn-malik-gigi-h...,hollywoodlife.com,7,0
23193,Jessica Chastain Recalls the Moment Her Mother...,http://www.justjared.com/2018/01/17/jessica-ch...,www.justjared.com,26,1
23194,"Tristan Thompson Feels ""Dumped"" After Khloé Ka...",www.intouchweekly.com/posts/tristan-thompson-f...,www.intouchweekly.com,24,0


In [104]:
# Get the independent feature
x = df.drop('real', axis=1)

In [105]:
#Get the dependent feature
y=df['real']

In [106]:
x.shape

(23196, 4)

In [107]:
y.shape

(23196,)

In [108]:
!pip install tensorflow

In [109]:
import tensorflow as tf

In [110]:
tf.__version__

'2.15.0'

In [111]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense


In [112]:
##Vocabulary size
voc_size = 5000

##Onehot Representation

In [113]:
messages = x.copy()

In [114]:
# title of the 1st index
messages['title'][1]

"People's Choice Awards 2018: The best red carpet looks"

In [115]:
messages

,title,news_url,source_domain,tweet_num
0,Kandi Burruss Explodes Over Rape Accusation on...,http://toofab.com/2017/05/08/real-housewives-a...,toofab.com,42
1,People's Choice Awards 2018: The best red carp...,https://www.today.com/style/see-people-s-choic...,www.today.com,0
2,Sophia Bush Sends Sweet Birthday Message to 'O...,https://www.etonline.com/news/220806_sophia_bu...,www.etonline.com,63
3,Colombian singer Maluma sparks rumours of inap...,https://www.dailymail.co.uk/news/article-33655...,www.dailymail.co.uk,20
4,Gossip Girl 10 Years Later: How Upper East Sid...,https://www.zerchoo.com/entertainment/gossip-g...,www.zerchoo.com,38
...,...,...,...,...
23191,Pippa Middleton wedding: In case you missed it...,https://www.express.co.uk/news/royal/807049/pi...,www.express.co.uk,52
23192,Zayn Malik & Gigi Hadid’s Shocking Split: Why ...,hollywoodlife.com/2018/03/13/zayn-malik-gigi-h...,hollywoodlife.com,7
23193,Jessica Chastain Recalls the Moment Her Mother...,http://www.justjared.com/2018/01/17/jessica-ch...,www.justjared.com,26
23194,"Tristan Thompson Feels ""Dumped"" After Khloé Ka...",www.intouchweekly.com/posts/tristan-thompson-f...,www.intouchweekly.com,24


In [116]:
messages.reset_index(inplace=True)

In [117]:
messages

,index,title,news_url,source_domain,tweet_num
0,0,Kandi Burruss Explodes Over Rape Accusation on...,http://toofab.com/2017/05/08/real-housewives-a...,toofab.com,42
1,1,People's Choice Awards 2018: The best red carp...,https://www.today.com/style/see-people-s-choic...,www.today.com,0
2,2,Sophia Bush Sends Sweet Birthday Message to 'O...,https://www.etonline.com/news/220806_sophia_bu...,www.etonline.com,63
3,3,Colombian singer Maluma sparks rumours of inap...,https://www.dailymail.co.uk/news/article-33655...,www.dailymail.co.uk,20
4,4,Gossip Girl 10 Years Later: How Upper East Sid...,https://www.zerchoo.com/entertainment/gossip-g...,www.zerchoo.com,38
...,...,...,...,...,...
23191,23191,Pippa Middleton wedding: In case you missed it...,https://www.express.co.uk/news/royal/807049/pi...,www.express.co.uk,52
23192,23192,Zayn Malik & Gigi Hadid’s Shocking Split: Why ...,hollywoodlife.com/2018/03/13/zayn-malik-gigi-h...,hollywoodlife.com,7
23193,23193,Jessica Chastain Recalls the Moment Her Mother...,http://www.justjared.com/2018/01/17/jessica-ch...,www.justjared.com,26
23194,23194,"Tristan Thompson Feels ""Dumped"" After Khloé Ka...",www.intouchweekly.com/posts/tristan-thompson-f...,www.intouchweekly.com,24


In [118]:
messages.shape

(23196, 5)

In [119]:
import nltk
import re
from nltk.corpus import stopwords

In [120]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [121]:
#Dataset Preprocessing
from nltk.stem import PorterStemmer # Stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
  review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
  review = review.lower()
  review = review.split()

  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [122]:
corpus

['kandi burruss explod rape accus real housew atlanta reunion video',
 'peopl choic award best red carpet look',
 'sophia bush send sweet birthday messag one tree hill co star hilari burton breyton eva',
 'colombian singer maluma spark rumour inappropri relationship aunt',
 'gossip girl year later upper east sider shock world chang pop cultur forev',
 'gwen stefani got dump blake shelton jealousi drama exclus',
 'broward counti sheriff fire lie parkland',
 'amber rose shut french montana date rumor call rapper bruvaaa',
 'mindi kale make first post babi appear disneyland wrinkl time co star',
 'katharin mcphee butcher toni nomin drink',
 'wag miami star ashley nicol robert philip wheeler marri',
 'mel gibson hollywood pedophil nowher left hide',
 'medium tyler henri address chill messag kristin cavallari deceas brother express read',
 'dwt season result week disney night',
 'via cq subscrib tax foundat',
 'reason tarek el moussa overcom latest back injuri',
 'david cassidi cut estrang 

In [123]:
corpus[0]

'kandi burruss explod rape accus real housew atlanta reunion video'

In [129]:
onehot_rep=[one_hot(words,voc_size)for words in corpus]
onehot_rep

[[4600, 682, 897, 3635, 2661, 556, 4803, 4669, 1589, 303],
 [4986, 2972, 2357, 174, 683, 1507, 3420],
 [2818,
  2716,
  1344,
  1917,
  3266,
  3125,
  3595,
  1063,
  1562,
  4852,
  3455,
  4993,
  1450,
  825,
  807],
 [532, 2836, 116, 2751, 1102, 2077, 992, 1365],
 [2182, 3329, 3569, 2181, 2660, 754, 565, 3170, 2277, 2678, 875, 2123, 3027],
 [2922, 1829, 3249, 3154, 2417, 2844, 1220, 4015, 4276],
 [2969, 525, 2596, 2270, 3623, 135],
 [3000, 2777, 3545, 2329, 2955, 1608, 3522, 3189, 2426, 4717],
 [4915, 811, 4806, 2238, 1011, 3148, 1168, 2429, 4333, 1327, 4852, 3455],
 [2996, 95, 4066, 561, 1739, 2000],
 [199, 1138, 3455, 4877, 2937, 2080, 4143, 2936, 3742],
 [3172, 790, 1866, 3758, 1518, 4810, 4130],
 [3675, 2808, 3606, 730, 802, 3125, 2925, 402, 1500, 3293, 3551, 3850],
 [1337, 1078, 4278, 264, 1198, 3478],
 [1281, 768, 4801, 4092, 858],
 [883, 403, 478, 2888, 405, 2751, 4627, 4566],
 [2864, 999, 4840, 4787, 4382, 2844, 2487, 1652, 4329, 1906],
 [1336, 1565, 4947, 1064, 750, 1844,

In [130]:
onehot_rep[0]

[4600, 682, 897, 3635, 2661, 556, 4803, 4669, 1589, 303]

##Embedding Representation

In [132]:
sent_length = 20 #Max length of he sentences
embedded_docs = pad_sequences(onehot_rep, padding = 'pre', maxlen = sent_length)
print(embedded_docs)

[[   0    0    0 ... 4669 1589  303]
 [   0    0    0 ...  683 1507 3420]
 [   0    0    0 ... 1450  825  807]
 ...
 [   0    0    0 ... 1833   51 4276]
 [   0    0    0 ... 4493 2519 2357]
 [   0    0    0 ... 1216 4756 1056]]


In [134]:
print(embedded_docs[0])

[   0    0    0    0    0    0    0    0    0    0 4600  682  897 3635
 2661  556 4803 4669 1589  303]


In [136]:
print(embedded_docs[1])

[   0    0    0    0    0    0    0    0    0    0    0    0    0 4986
 2972 2357  174  683 1507 3420]


In [138]:
#Creating model
embedding_vector_features = 40 #Features representation
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length = sent_length)) #Add the embedding layer
model.add(LSTM(100)) #Add LSTM with 100 nuerons
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss ='binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_1 (LSTM)               (None, 100)               56400     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
